In [1]:
import json
import os
import uuid
import random
from typing import TypedDict, List, Optional, Dict
from collections import defaultdict

from langchain_groq import ChatGroq
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv

c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
load_dotenv()

groq_key = os.getenv("GROQ_API_KEY")
if groq_key is None:
	raise RuntimeError("GROQ_API_KEY environment variable not set")
os.environ["GROQ_API_KEY"] = groq_key

In [3]:
llm = ChatGroq(model="llama-3.1-8b-instant")
checkpointer = MemorySaver()

In [4]:
def clean_json_string(json_str):
    if "```json" in json_str:
        json_str = json_str.split("```json")[1].split("```")[0]
    elif "```" in json_str:
        json_str = json_str.split("```")[1].split("```")[0]
    return json_str.strip()

In [5]:
class State(TypedDict):
    role: str
    difficulty_pools: Dict[str, List[dict]]
    current_difficulty: str
    max_questions: int
    questions_asked: int
    current_question: Optional[str]
    current_area: Optional[str]
    current_diff: Optional[str]
    scores: List[int]
    areas: List[str]
    difficulties: List[str]
    feedbacks: List[str]
    transcript: List[str]
    user_answer: Optional[str]
    feedback: Optional[str]
    score: Optional[int]
    follow_up: Optional[str]
    follow_up_count: int
    is_end: bool

In [6]:
def prepare_question(state: State) -> State:
    if state["is_end"]:
        return state

    if state["follow_up"] and state.get("follow_up_count", 0) < 1:
        state["current_question"] = state["follow_up"]
        state["follow_up"] = None
        state["follow_up_count"] = state.get("follow_up_count", 0) + 1
    else:
        state["follow_up_count"] = 0 
        
        if state["questions_asked"] >= state["max_questions"]:
            state["is_end"] = True
            return state
            
        level = state["current_difficulty"]
        pools = state["difficulty_pools"]
        
        if not pools[level]:
            if level == "hard" and pools["medium"]: level = "medium"
            elif level == "medium" and pools["easy"]: level = "easy"
            elif level == "easy" and pools["hard"]: level = "hard"
            else:
                state["is_end"] = True
                return state
            state["current_difficulty"] = level

        q_list = pools[level]
        random.shuffle(q_list)
        q_dict = q_list.pop()
        
        state["current_question"] = q_dict["text"]
        state["current_area"] = q_dict["area"]
        state["current_diff"] = level
        state["questions_asked"] += 1
        
    return state

In [7]:
def human_feedback(state: State) -> State:
    return state

In [8]:
def evaluate(state: State) -> State:
    q = state["current_question"]
    a = state["user_answer"] or ""

    if not a.strip():
        return {"feedback": "No answer provided.", "score": 0, "follow_up": None}
    if a.lower().strip() == "skip":
        return {"score": 0, "feedback": "Skipped.", "follow_up": None}
    if a.lower().strip() in ["quit", "exit", "stop"]:
        return {"is_end": True, "score": None, "feedback": "Interview terminated.", "follow_up": None}

    eval_prompt = f"""
    You are a technical interviewer for the role of {state['role']}.
    Current Question: {q}
    Candidate Answer: {a}

    Evaluate the answer strictly.
    Output JSON: {{"score": <0-100>, "feedback": "<Brief critique>", "follow_up": "<string or null>"}}
    """
    
    try:
        resp = llm.invoke(eval_prompt, response_format={"type": "json_object"}).content
        data = json.loads(clean_json_string(resp))
        score = data["score"]
        fb = data["feedback"]
        fu = data.get("follow_up", None)
    except:
        score = 0
        fb = "Error evaluating."
        fu = None
        
    return {"score": score, "feedback": fb, "follow_up": fu}

In [9]:
def adjust_difficulty(state: State) -> State:
    if state["score"] is not None:
        score = state["score"]
        current = state["current_difficulty"]
        if score >= 85 and current != "hard":
            state["current_difficulty"] = "hard" if current == "medium" else "medium"
        elif score < 50 and current != "easy":
            state["current_difficulty"] = "medium" if current == "hard" else "easy"
    return state

In [10]:
def process(state: State) -> State:
    if state["score"] is not None:
        state["scores"].append(state["score"])
        state["areas"].append(state["current_area"])
        state["difficulties"].append(state["current_diff"])
        state["feedbacks"].append(state["feedback"])
        
        qa_entry = f"Q: {state['current_question']}\nA: {state['user_answer']}\nScore: {state['score']}\nFeedback: {state['feedback']}"
        state["transcript"].append(qa_entry)
    
    state["user_answer"] = None
    state["score"] = None
    state["feedback"] = None
    return state

In [11]:
def decide(state: State) -> str:
    if state["is_end"]:
        return END
    return "prepare_question"

In [12]:
builder = StateGraph(State)
builder.add_node("prepare_question", prepare_question)
builder.add_node("human_feedback", human_feedback)
builder.add_node("evaluate", evaluate)
builder.add_node("adjust_difficulty", adjust_difficulty)
builder.add_node("process", process)

builder.add_edge(START, "prepare_question")
builder.add_edge("prepare_question", "human_feedback")
builder.add_edge("human_feedback", "evaluate")
builder.add_edge("evaluate", "adjust_difficulty")
builder.add_edge("adjust_difficulty", "process")
builder.add_conditional_edges("process", decide, {END: END, "prepare_question": "prepare_question"})

app = builder.compile(checkpointer=checkpointer, interrupt_before=["human_feedback"])

def generate_final_report(state: Dict):
    if not state.get("transcript"):
        print("No interview data to analyze.")
        return

    print("\n" + "="*50, flush=True)
    print("📝 GENERATING DETAILED PERFORMANCE REPORT...", flush=True)
    print("="*50, flush=True)

    transcript_text = "\n---\n".join(state["transcript"])
    
    report_prompt = f"""
    You are an Expert Interview Coach. Review the following interview transcript for a {state['role']} role.
    
    TRANSCRIPT:
    {transcript_text}
    
    Generate a structured feedback report in markdown format covering:
    1. **Executive Summary**: A 2-sentence overview of performance.
    2. **Technical Knowledge**: Strengths and gaps observed.
    3. **Communication Style**: Clarity, confidence, and structure of answers.
    4. **Key Areas for Improvement**: 3 specific actionable tips.
    5. **Final Verdict**: (Strong Hire / Hire / Weak Hire / No Hire).
    
    Keep it professional and constructive.
    """
    
    try:
        report = llm.invoke(report_prompt).content
        print(report, flush=True)
    except Exception as e:
        print(f"Error generating report: {e}")

def run_interview():
    thread_id = str(uuid.uuid4())
    config = {"configurable": {"thread_id": thread_id}}

    role = input("Enter job role: ")
    print(f"Generating questions for {role}...", flush=True)

    try:
        gen_prompt = f"Generate 8 interview questions for {role} in JSON format: {{'questions': [{{'text': '...', 'area': 'Technical/Behavioral', 'difficulty': 'easy/medium/hard'}}]}}"
        resp = llm.invoke(gen_prompt).content
        data = json.loads(clean_json_string(resp))
        questions = data["questions"]
    except:
        questions = [
            {"text": "Tell me about yourself.", "area": "Behavioral", "difficulty": "easy"},
            {"text": "What is your biggest strength?", "area": "Behavioral", "difficulty": "medium"},
            {"text": "Explain a technical concept you know well.", "area": "Technical", "difficulty": "medium"}
        ]

    pools = {"easy": [], "medium": [], "hard": []}
    for q in questions:
        d = q.get("difficulty", "medium").lower()
        if d in pools: pools[d].append(q)

    initial_state = {
        "role": role,
        "difficulty_pools": pools,
        "current_difficulty": "medium",
        "max_questions": 5,
        "questions_asked": 0,
        "current_question": None,
        "current_area": None,
        "current_diff": None,
        "scores": [],
        "areas": [],
        "difficulties": [],
        "feedbacks": [],
        "transcript": [],
        "user_answer": None,
        "feedback": None,
        "score": None,
        "follow_up": None,
        "follow_up_count": 0,
        "is_end": False
    }

    events = app.stream(initial_state, config)
    for event in events: pass 

    print(f"\n🤖 INTERVIEW START: {role}", flush=True)

    while True:
        snapshot = app.get_state(config)
        if not snapshot.next:
            break
        
        current_q = snapshot.values.get('current_question')
        if not current_q:
            break
            
        print(f"\n🤖 Question: {current_q}", flush=True)
        user_input = input("You: ")
        
        if user_input.lower() in ["quit", "exit"]:
            break

        app.update_state(config, {"user_answer": user_input}, as_node="human_feedback")
        print("   ...(Analyzing)...", flush=True)
        
        try:
            events = app.stream(None, config)
            for event in events:
                if 'process' in event and event['process'].get('feedbacks'):
                    print(f"   (Immediate Feedback: {event['process']['feedbacks'][-1]})", flush=True)
        except Exception as e:
            print(f"Error: {e}")
            break

    final_state = app.get_state(config).values
    scores = final_state.get("scores", [])
    
    if scores:
        avg_score = sum(scores) / len(scores)
        main_qs = final_state["questions_asked"]
        total_turns = len(scores)
        
        print("\n" + "="*50, flush=True)
        print(f"📊 STATISTICS", flush=True)
        print(f"Main Questions Covered: {main_qs} / {final_state['max_questions']}")
        print(f"Total Interactions (including follow-ups): {total_turns}")
        print(f"Average Proficiency Score: {avg_score:.1f}/100")
        
        generate_final_report(final_state)
    else:
        print("No questions answered.")

In [13]:
if __name__ == "__main__":
    run_interview()

Generating questions for data analyst...

🤖 INTERVIEW START: data analyst

🤖 Question: What is your experience with data visualization tools such as Tableau or Power BI? Can you give an example of a dashboard you've created?
   ...(Analyzing)...
   (Immediate Feedback: No answer provided.)

🤖 Question: How do you stay up-to-date with new tools and technologies in the field of data analysis?
   ...(Analyzing)...
   (Immediate Feedback: No answer provided.)

🤖 Question: How do you handle missing or inconsistent data in a dataset?
   ...(Analyzing)...
   (Immediate Feedback: No answer provided.)

🤖 Question: What statistical methods would you use to determine the correlation between two variables?
   ...(Analyzing)...
   (Immediate Feedback: No answer provided.)

🤖 Question: What is SQL, and how do you use it to query and analyze data?
   ...(Analyzing)...
   (Immediate Feedback: No answer provided.)

🤖 Question: What is SQL, and how do you use it to query and analyze data?
   ...(Analyzi